<a href="https://colab.research.google.com/github/Thedatababbler/CVPR2021-Papers-with-Code/blob/master/nb/Llama3.1_(8B)-GRPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [Gemma 3 blog](https://unsloth.ai/blog/gemma3) for what's new in Unsloth and our [Reasoning blog](https://unsloth.ai/blog/r1-reasoning) on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [3]:
%%capture
# Skip restarting message in Colab
import sys; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

!pip install unsloth vllm
!pip install --upgrade pillow

In [8]:
!pip install numpy==1.25

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 100.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
blosc2 3.2.0 requires numpy>=1.26, but you have numpy 1.25.0 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.25.0 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.25.0 which is incompatible.


### Unsloth

Load up `Llama 3.1 8B Instruct`, and set parameters

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "meta-llama/meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.6, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

==((====))==  Unsloth 2025.3.17: Fast Llama patching. Transformers: 4.48.3. vLLM: 0.8.1.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/meta-llama-3.1-8b-instruct-unsloth-bnb-4bit with actual GPU utilization = 20.19%
Unsloth: Your GPU has CUDA compute capability 8.0 with VRAM = 39.56 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 1024. Num Sequences = 128.
Unsloth: vLLM's KV Cache can use up to 1.81 GB. Also swap space = 6 GB.
INFO 03-20 17:48:00 [config.py:583] This model supports multiple tasks: {'embed', 'reward', 'score', 'classify', 'generate'}. Defaulting to 'generate'.
Unsloth: vLLM Bitsan

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


### Data Prep
<a name="Data"></a>

We directly leverage [@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) for data prep and all reward functions. You are free to create your own!

In [2]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train") -> Dataset:
    data = load_dataset('openai/gsm8k', 'main')[split] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    }) # type: ignore
    return data # type: ignore

dataset = get_gsm8k_questions()

# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

In [5]:
!pip install datasets

In [22]:
# prompt: download a dataset from huggingface and load it


SYSTEM_PROMPT = """
You are a helpful assistant that helps users answer questions.
You will be given a bunch of medical exam questions with 4 options.
Your task is to select the correct option and respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
Note: You should only respond with the selected option in the answer section.
Please strictly follow the format.
"""

XML_COT_FORMAT = """\
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""


import sys; modules = list(sys.modules.keys())
# from unsloth import FastLanguageModel
# import torch
# import re
from datasets import load_dataset, Dataset

# def process_p
# uncomment middle messages for 1-shot prompting
def get_function(split = "train") -> Dataset:
    data = load_dataset('GBaker/MedQA-USMLE-4-options')[split] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']+'\tOptions:'+'; '.join([f'{k}: {v}' for k, v in x['options'].items()])}
        ],
        'org_answer': x['answer'],
        'answer': x['answer_idx']

    }) # type: ignore
    return data # type: ignore

dataset = get_function()
dataset[1]

Map:   0%|          | 0/10178 [00:00<?, ? examples/s]

{'question': 'A 3-month-old baby died suddenly at night while asleep. His mother noticed that he had died only after she awoke in the morning. No cause of death was determined based on the autopsy. Which of the following precautions could have prevented the death of the baby?',
 'answer': 'A',
 'options': {'A': 'Placing the infant in a supine position on a firm mattress while sleeping',
  'B': 'Keeping the infant covered and maintaining a high room temperature',
  'C': 'Application of a device to maintain the sleeping position',
  'D': 'Avoiding pacifier use during sleep'},
 'meta_info': 'step2&3',
 'answer_idx': 'A',
 'metamap_phrases': ['3 month old baby died',
  'night',
  'asleep',
  'mother',
  'died only',
  'awoke in',
  'morning',
  'cause of death',
  'based',
  'autopsy',
  'following precautions',
  'prevented',
  'death',
  'baby'],
 'prompt': [{'content': '\nYou are a helpful assistant that helps users answer questions.\nYou will be given a bunch of medical exam questions 

<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [9]:
!wandb login e4ba3be8a795947e787d1344e6898362060309b9

wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [12]:
import wandb
wandb.init(project='grpo_unsloth')

In [16]:
max_seq_length

1024

In [31]:
max_prompt_length = 512


from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 6, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 300,
    save_steps = 200,
    max_grad_norm = 0.1,
    report_to = "wandb", # Can use Weights & Biases
    output_dir = "outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 6


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [32]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10,178 | Num Epochs = 1 | Total steps = 300
O^O/ \_/ \    Batch size per device = 6 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (6 x 1 x 1) = 6
 "-____-"     Trainable parameters = 83,886,080/8,000,000,000 (1.05% trained)


-------------------- Question:
A 17-year-old male, accompanied by his uncle, presents to his family physician with his arm in a sling. There is blood dripping down his shirt. He pleads with the physician to keep this injury "off the books", offering to pay in cash for his visit, as he is afraid of retaliation from his rival gang. The physician exams the wound, which appears to be a stabbing injury to his left anterior deltoid. How should the physician best handle this patient's request?	Options:A: Maintain confidentiality, as retaliation may result in greater harm to the patient; B: Maintain confidentiality, as reporting stab wounds is not required; C: Maintain confidentiality and schedule a follow-up visit with the patient; D: Breach confidentiality and report the stab wound to the police 
Answer:
D 
Response:
<reasoning>
The vaccine given to the newborn is most likely the diphtheria, tetanus, and pertussis vaccine (DTaP), which is traditionally administered at birth as part of a seri

Step,Training Loss,reward,reward_std,completion_length,kl,rewards / xmlcount_reward_func,rewards / soft_format_reward_func,rewards / strict_format_reward_func,rewards / int_reward_func,rewards / correctness_reward_func
1,0.105700,0.187500,0.104583,223.500000,2.641618,0.187500,0.000000,0.000000,0.000000,0.000000
2,0.001700,2.250000,0.000000,203.000000,0.041632,0.250000,0.000000,0.000000,0.000000,2.000000
3,0.062800,0.541667,0.839146,208.666672,1.570279,0.208333,0.000000,0.000000,0.000000,0.333333
4,0.040000,0.895000,1.050667,133.166672,1.000431,0.228333,0.000000,0.000000,0.000000,0.666667
5,0.002100,2.250000,0.000000,179.000000,0.053495,0.250000,0.000000,0.000000,0.000000,2.000000
6,0.018500,1.229333,1.124617,160.833344,0.461313,0.229333,0.000000,0.000000,0.000000,1.000000
7,0.039600,1.562500,1.065804,203.833344,0.989395,0.229167,0.000000,0.000000,0.000000,1.333333
8,0.006700,1.246667,1.101507,204.333344,0.166625,0.246667,0.000000,0.000000,0.000000,1.000000
9,0.001900,0.583333,0.816497,186.500000,0.048670,0.250000,0.000000,0.000000,0.000000,0.333333
10,0.002300,1.916667,0.816497,138.833344,0.057618,0.250000,0.000000,0.000000,0.000000,1.666667


Streaming output truncated to the last 5000 lines.
 —#B&#;/ 
Extracted:

-------------------- Question:
A 42-year-old man presents to his primary care provider for a follow-up appointment after a new diagnosis of hypertension follow-up. The doctor mentions that a recent study where the effect of a healthy lifestyle education program on blood pressure was studied in 2 matched rural communities. One community received health education program and the other did not. What is the type of study most likely being described here?	Options:A: Case-control trial; B: Explanatory study; C: Community trial; D: Cross-sectional study 
Answer:
C 
Response:
<reasoning>
The question is describing a scenario where two groups, one receiving an intervention (health education program) and the other not, are being compared. This type of study design is typically used to test the effect of an intervention or treatment on an outcome, in this case, blood pressure. The fact that there are two matched rural commun

TrainOutput(global_step=300, training_loss=0.03357143026427366, metrics={'train_runtime': 4064.7448, 'train_samples_per_second': 0.443, 'train_steps_per_second': 0.074, 'total_flos': 0.0, 'train_loss': 0.03357143026427366})

<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:51<00:00, 51.81s/it, est. speed input: 0.75 toks/s, output: 17.01 toks/s]


'**Calculating Pi using Python**\n\nPi (π) is a mathematical constant representing the ratio of a circle\'s circumference to its diameter. Here\'s a simple and efficient way to calculate an approximation of pi using Python.\n\n### Using the Monte Carlo Method\n\nThe Monte Carlo method is a computational algorithm that uses random sampling to approximate a value. In this case, we can use it to estimate pi by generating random points within a square and checking if they fall inside a quarter-circle inscribed within it.\n\n```python\nimport random\nimport math\n\ndef estimate_pi(num_samples):\n    """\n    Estimate the value of pi using the Monte Carlo method.\n\n    Args:\n    num_samples (int): The number of random points to generate.\n\n    Returns:\n    float: An approximation of pi.\n    """\n    points_inside_circle = 0\n\n    for _ in range(num_samples):\n        x, y = random.random(), random.random()\n        distance = x**2 + y**2\n        if distance <= 1:\n            points_i

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [25]:
model.save_lora("grpo_saved_lora")

Now we load the LoRA and test:

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [33]:
# prompt: save peft  model to my google drive

model.save_lora("/content/drive/My Drive/grpo_saved_lora2")


In [34]:


SYSTEM_PROMPT = """
You are a helpful assistant that helps users answer questions.
You will be given a bunch of medical exam questions with 4 options.
Your task is to select the correct option and respond in the following format:
<answer>
...
</answer>
Note: You should only respond the final option Letter in the answer section.
Please strictly follow the format.
"""

# def process_p
# uncomment middle messages for 1-shot prompting
def get_function(split = "train") -> Dataset:
    data = load_dataset('GBaker/MedQA-USMLE-4-options')[split] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']+'\tOptions:'+'; '.join([f'{k}: {v}' for k, v in x['options'].items()])}
        ],
        'org_answer': x['answer'],
        'answer': x['answer_idx']

    }) # type: ignore
    return data # type: ignore

test_set = get_function('test')
test_set[1]

Map:   0%|          | 0/1273 [00:00<?, ? examples/s]

{'question': "A 67-year-old man with transitional cell carcinoma of the bladder comes to the physician because of a 2-day history of ringing sensation in his ear. He received this first course of neoadjuvant chemotherapy 1 week ago. Pure tone audiometry shows a sensorineural hearing loss of 45 dB. The expected beneficial effect of the drug that caused this patient's symptoms is most likely due to which of the following actions?",
 'answer': 'D',
 'options': {'A': 'Inhibition of proteasome',
  'B': 'Hyperstabilization of microtubules',
  'C': 'Generation of free radicals',
  'D': 'Cross-linking of DNA'},
 'meta_info': 'step1',
 'answer_idx': 'D',
 'metamap_phrases': ['67 year old man',
  'transitional cell carcinoma of the bladder',
  'physician',
  '2-day history',
  'ringing sensation',
  'ear',
  'received',
  'first course',
  'neoadjuvant chemotherapy 1 week',
  'Pure tone audiometry shows',
  'sensorineural hearing loss',
  'dB',
  'expected beneficial effect',
  'drug',
  'caused

In [35]:
from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.2,
    top_p = 0.95,
    max_tokens = 1024,
)



In [36]:
acc_list = []
# from pprint import pprint
for entry in test_set:
  # pprint(entry['prompt'])
  text = tokenizer.apply_chat_template(entry['prompt'], tokenize = False, add_generation_prompt = True)
  # text = tokenizer.apply_chat_template([
  #     {"role" : "user", "content" : "Calculate pi."},
  # ], tokenize = False, add_generation_prompt = True)
  output = model.fast_generate(
      [text],
      sampling_params = sampling_params,
      lora_request = None,
  )[0].outputs[0].text

  final_answer = extract_xml_answer(output)
  print(f'Extracted_anser:{final_answer}')
  print(f'Org_answer:{entry["answer"]}')
  acc_list.append(final_answer == entry["answer"])

print(f'Accuracy:{sum(acc_list)/len(acc_list)}')
#

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  3.25it/s, est. speed input: 883.64 toks/s, output: 19.63 toks/s]


Extracted_anser:D
Org_answer:B


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  4.42it/s, est. speed input: 1020.22 toks/s, output: 26.73 toks/s]


Extracted_anser:D
Org_answer:D


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  4.94it/s, est. speed input: 2037.28 toks/s, output: 24.90 toks/s]


...

Extracted_anser:D
Org_answer:B


Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.18s/it, est. speed input: 270.03 toks/s, output: 67.08 toks/s]


Extracted_anser:C
Org_answer:C


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  4.63it/s, est. speed input: 1387.48 toks/s, output: 28.03 toks/s]

Extracted_anser:C
Org_answer:C
Accuracy:0.3558523173605656


In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:23<00:00, 23.17s/it, est. speed input: 2.63 toks/s, output: 15.80 toks/s]


'Calculating pi to a high degree of accuracy is a complex task that requires a large amount of computational power. However, I can provide you with an approximate value of pi or show you a simple method to calculate it.\n\nOne of the simplest methods to calculate pi is the Leibniz formula, which is an infinite series:\n\npi/4 = 1 - 1/3 + 1/5 - 1/7 + 1/9 - ...\n\nThis series can be used to calculate an approximation of pi.\n\n<reasoning>\npi = 4 * (1 - 1/3 + 1/5 - 1/7 + 1/9 - ...)\n</reasoning>\n\nThis is a simple, yet effective method to calculate pi. However, the more terms you use, the more accurate the result will be.\n\nTo calculate pi to a high degree of accuracy, you would need to use a computer program to perform the calculation.\n\n<answer>\n3.141592653589793 (approximately)\n</answer>\n\nFor a more accurate result, I can provide you with a Python code snippet to calculate pi:\n\n```python\nimport math\n\ndef calculate_pi(n):\n    pi = 0.0\n    for i in range(n):\n        pi +=

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
